In [ ]:
"""Тестирование проводим за период с 18.12.2012 по 29.04.2022, 
т.к. с сайта Мосбиржи доступна подробная статистика по ребалансировкам для этого периода.
Можно сделать с 04.03.2011, но нет инф-ции по капитализации для тестирования фактора Size,
поэтому будут только Momentum, Low volatility и High dividends"""

***Создание свода о ребалансировках наших индексов + цены с дивидендами***

In [ ]:
# Добавить комиссии за ребалансировку!

In [291]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [292]:
rebalancing = pd.read_excel('moex - assets.xlsx', sheet_name='динамика для бэктеста', header=1)
price = pd.read_excel('moex - assets.xlsx', sheet_name='цены', index_col=0).iloc[1:]
price = price.fillna(0)
dividends = pd.read_excel('moex - assets.xlsx', sheet_name='дивиденды').iloc[:, 1:]
Nshares = pd.read_excel('moex - assets.xlsx', sheet_name='количество акций', index_col=0)

In [293]:
# Функция берет столбец с ценами и возвращает столбец с изменениями цен без учета неторгуемых дней
def price_to_changes(price_path):
    price_path_f = price_path.copy()
    price_path_f = price_path_f[price_path_f != 0]
    price_changes = price_path_f / price_path_f.shift(1) - 1
    price_changes = price_changes.dropna()
    price_changes = price_changes[price_changes != 0]
    
    return(price_changes)

In [294]:
# Функция берет дивидендную дату и, если дивиденд есть и ГЭП попадает в динамику цен, то находит примерную дату, когда был дивидендный ГЭП
def div_date_to_gap_date(div, div_date, price_changes):
    if not(np.isnan(div)):
        if (div_date >= price_changes.index[2]) and (div_date <= price_changes.index[-1]):
            div_date = price_changes[price_changes.index <= div_date].index[-1]
            possible_gap = price_changes[price_changes.index <= div_date][-3:]
            div_gap_date = possible_gap[possible_gap == possible_gap.min()].index[0]
        else:
            div_gap_date = 'За рамками динамики'
    else:
        div_gap_date = 'Нет дивиденда'
        
    return(div_gap_date)

In [295]:
# Функция добавляет в словарь параметры Моментума с указанным окном
# 'Недостаточная история торгов' - если не хватает истории котировок как минимум на половину окна моментума

def momentum_calc(selection, price, windowM):
    selection_f = selection.copy()
    windowM = 365 // windowM
    for date in selection_f.keys():
        param_calc = []

        for ticker in selection_f[date]['Актив']:
            dynamics = price[ticker]
            dynamics = dynamics[dynamics.index < date]
            dynamics = dynamics[dynamics.index > date - timedelta(windowM)]
            dynamics = dynamics[dynamics != 0]
            dif = (date - dynamics.index[0]).days

            if dif < windowM // 2:
                param_calc.append('Недостаточная история торгов')
            else:
                change = dynamics[-1] / dynamics[0] - 1
                param_calc.append(change)

        selection_f[date]['Momentum_' + str(int(12 / (365 // windowM))) + "M"] = param_calc
    
    return(selection_f)

In [296]:
# Добавим в цену дивиденды. Все тикеры из таблицы с ценами должны быть в таблице с дивидендами, иначе выдаст ошибку.
tickers = price.columns.tolist()
for ticker in tickers:
    divs = dividends[['Date ' + ticker, 'Divs ' + ticker]]
    price_path = price[ticker].copy()
    price_changes = price_to_changes(price_path)
    for i in range(len(divs)):
        div_date = divs.iloc[i, 0]
        div = divs.iloc[i, 1]
        div_gap_date = div_date_to_gap_date(div, div_date, price_changes)
        if div_gap_date == 'Нет дивиденда':
            break
        if div_gap_date == 'За рамками динамики':
            continue
        else:
            price_path[price_path.index == div_gap_date] += div
    price[ticker] = price_path

In [297]:
# Частота ребалансировки (в месяцах)
freq = 3
freq_days = freq * 30
# Указываем окно для бэктеста (динамика)
# start_date должна быть на freq месяцев больше, чем минимальная дата в котировках
start_date = pd.to_datetime('18.12.2012')
end_date = pd.to_datetime('29.04.2022')

In [298]:
last_reb_date = start_date - timedelta(freq_days)
ac_reb_date = start_date
# min_days - столько дней должны торговаться половина компаний из индекса с момента последней ребалансировки
min_days = round(freq * 30 * 5 / 7 * 0.3)

In [299]:
# В словарь moex_rebs запишем по датам ребалансировок из нашего периода актуальные акции - Universe
universe1 = rebalancing.columns[(rebalancing.columns <= start_date).sum() - 1]
moex_rebs = {universe1 : rebalancing[universe1][[x in tickers for x in rebalancing[universe1]]].tolist()}
reb_dates = rebalancing.columns[(rebalancing.columns > start_date) & (rebalancing.columns < end_date)]
for reb_date in reb_dates:
    moex_rebs[reb_date] = rebalancing[reb_date][[x in tickers for x in rebalancing[reb_date]]].tolist()

In [300]:
# Ребалансировки индекса со списком активов, которые торговались в дату ребалансировки
real_rebs = {}
while ac_reb_date < end_date:

    moex_date = list(moex_rebs.keys())[sum([x <= ac_reb_date for x in list(moex_rebs.keys())]) - 1]
    ac_universe = moex_rebs[moex_date]
    not_trading = []

    Ntrade = 0
    Ntrade_last = 0
    for ticker in ac_universe:
        price_universe = price[ticker][price[ticker] != 0]
        price_universe = price_universe[price_universe.index <= ac_reb_date]
        price_universe = price_universe[price_universe.index >= last_reb_date]
        change_universe = (price_universe / price_universe.shift(1)).dropna()
        trade_days = (change_universe != 1).sum()
        if trade_days >= min_days:
            Ntrade += 1
        if change_universe[-1] != 1:
            Ntrade_last += 1
        else:
            not_trading.append(ticker)

    if Ntrade / len(ac_universe) >= 0.5:
        if Ntrade_last / len(ac_universe) >= 0.5:
            ac_universe = [x for x in ac_universe if x not in not_trading]
            real_rebs[ac_reb_date] = ac_universe
            last_reb_date = ac_reb_date
            ac_reb_date = ac_reb_date + timedelta(freq_days)
        else:
            ac_reb_date = ac_reb_date + timedelta(1)
    
    else:
        ac_reb_date = ac_reb_date + timedelta(freq_days)


***Расчет значений стратегий***

In [301]:
# В каждую дату ребалансировки положим датафрейм, куда будем складывать параметры стратегий через функции
selection = {}
for i in real_rebs.keys():
    selection[i] = pd.DataFrame(real_rebs[i], columns=['Актив'])

In [302]:
# Добавляем Моментум  (можно несколько раз с разными окнами)
# windowM - кол-во ребалансировок в год
windowM = 4
selection = momentum_calc(selection, price, windowM)

In [303]:
selection

{Timestamp('2012-12-18 00:00:00'):     Актив  Momentum_3M
 0    AFKS    -0.056783
 1    AFLT    -0.021358
 2    AKRN    -0.047919
 3    ALRS     0.091680
 4    BANE    -0.029159
 5   BANEP     0.031428
 6    CHMF    -0.126539
 7    UPRO    -0.064952
 8    FEES    -0.107937
 9    GAZP    -0.151866
 10   GMKN     0.105367
 11   HYDR    -0.167738
 12   IRAO    -0.100036
 13   LKOH     0.025834
 14   LSRG    -0.036256
 15   MAGN    -0.092637
 16   MGNT     0.139759
 17   RSTI    -0.149952
 18   MSNG    -0.029836
 19   MSRS     0.214115
 20   MSTT    -0.011244
 21   MTLR    -0.122594
 22  MTLRP    -0.255673
 23   MTSS     0.023355
 24   MVID    -0.134180
 25   NLMK    -0.066707
 26   NVTK    -0.029273
 27   OGKB    -0.169031
 28   PHOR     0.210725
 29   PIKK    -0.189272
 30   RASP    -0.235603
 31   ROSN     0.251029
 32   RTKM    -0.133694
 33  RTKMP    -0.067054
 34   SBER    -0.002559
 35  SBERP    -0.046230
 36   SNGS    -0.085662
 37  SNGSP     0.001765
 38   SVAV     0.193273
 39   

In [ ]:
!!!

In [ ]:
'''Попробуем следующие варианты:
Моментум - 3 и 6 месяцев
Волатильность - 3, 6, 12 месяцев'''
windowM = 3
windowV = 3

windowM = 365 // windowM
windowV = 365 // windowV